In [1]:
%env CUDA_VISIBLE_DEVICES=1 TOKENIZERS_PARALLELISM=true

env: CUDA_VISIBLE_DEVICES=1 TOKENIZERS_PARALLELISM=true


### Task 1

In [2]:
from task1.train import train

In [3]:
train(with_amp=False, scaler_type=None)

Loss: 0.6165 Accuracy: 92.918: 100%|██████████| 40/40 [00:39<00:00,  1.02it/s] 


In [4]:
train(with_amp=True, scaler_type="static")

Loss: 0.5857 Accuracy: 98.6872: 100%|██████████| 40/40 [00:38<00:00,  1.05it/s]


In [5]:
train(with_amp=True, scaler_type="dynamic")

Loss: 0.5847 Accuracy: 98.7843: 100%|██████████| 40/40 [00:38<00:00,  1.05it/s]


### Task 2

In [2]:
from task2.run_epoch import run_epoch, DataMode

In [3]:
run_epoch(DataMode.BRAIN)

100%|██████████| 313/313 [01:16<00:00,  4.10it/s]


,pad_token_ratio,time
count,308.000000,308.000000
mean,0.848730,0.242300
std,0.036678,0.009445
min,0.705371,0.141561
25%,0.826880,0.237154
50%,0.848437,0.243845
75%,0.872119,0.249077
max,0.943945,0.254336


In [4]:
run_epoch(DataMode.BIG_BRAIN)

100%|██████████| 313/313 [00:37<00:00,  8.35it/s]


,pad_token_ratio,time
count,308.000000,308.000000
mean,0.656957,0.118676
std,0.076169,0.034983
min,0.384375,0.053899
25%,0.607784,0.094726
50%,0.668906,0.114410
75%,0.711805,0.132526
max,0.817412,0.275393


In [5]:
run_epoch(DataMode.ULTRA_BIG_BRAIN)

100%|██████████| 313/313 [00:37<00:00,  8.26it/s]


,pad_token_ratio,time
count,308.000000,308.000000
mean,0.659161,0.120097
std,0.081057,0.034258
min,0.404622,0.049049
25%,0.606761,0.097358
50%,0.668085,0.115656
75%,0.717234,0.136890
max,0.833252,0.273259


In [6]:
run_epoch(DataMode.ULTRA_BIG_BRAIN, k=40)

100%|██████████| 319/319 [00:14<00:00, 21.42it/s]


,pad_token_ratio,time
count,314.000000,314.000000
mean,0.305022,0.046824
std,0.237976,0.034199
min,0.000000,0.012199
25%,0.100979,0.016014
50%,0.177784,0.033884
75%,0.583558,0.064275
max,0.721875,0.157416


In [7]:
run_epoch(DataMode.ULTRA_BIG_BRAIN, k=20)

100%|██████████| 328/328 [00:14<00:00, 23.37it/s]


,pad_token_ratio,time
count,323.000000,323.000000
mean,0.188279,0.042896
std,0.168477,0.032377
min,0.000000,0.011761
25%,0.049851,0.012869
50%,0.096875,0.033257
75%,0.357738,0.063277
max,0.534375,0.150378


In [4]:
run_epoch(DataMode.ULTRA_BIG_BRAIN, k=5)

100%|██████████| 368/368 [00:12<00:00, 28.75it/s]


,pad_token_ratio,time
count,363.000000,363.000000
mean,0.060522,0.034351
std,0.069257,0.024848
min,0.000000,0.009408
25%,0.010329,0.012260
50%,0.020312,0.026308
75%,0.112500,0.049903
max,0.225000,0.113555


In [3]:
run_epoch(DataMode.ULTRA_DUPER_BIG_BRAIN)

,pad_token_ratio,time
count,657.0,658.000000
mean,0.0,0.017099
std,0.0,0.000705
min,0.0,0.005387
25%,0.0,0.016853
50%,0.0,0.017043
75%,0.0,0.017243
max,0.0,0.022839
